In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [7]:
import json
import pandas as pd
import os
import re
import string
import pickle

from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
import os
#os.chdir('../..')

from vecsim_app.categories import CATEGORIES
from vecsim_app.data_utils import papers

Define the parameters for fetching the papers dataset:

- Dataset Path
- Year cutoff: Year cut off for the papers.
- Pattern for fetching a given amount of years
- Max Sample size: maximum simple size (if you just want to try out the notebook - if it's too low the model won't perform well)

In [13]:
DATA_PATH = "./arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2010
YEAR_PATTERN = r"(19|20[0-9]{2})"
MAX_SAMPLE_SIZE = 5000


In [21]:
MAX_SAMPLE_SIZE = 100  # to get a sample dummy model

In [16]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN))
len(df)

# Take a sample for computing reasons
df = df.sample(MAX_SAMPLE_SIZE)

In [18]:
df.shape

(100, 6)

In [22]:
df.head(3)

,id,title,year,authors,categories,abstract,text
442317,2106.08442,Numerical transfer matrix study of frustrated ...,2021,Yi Hu and Patrick Charbonneau,"cond-mat.stat-mech,cond-mat.soft",Ising models with frustrated next-nearest-ne...,Numerical transfer matrix study of frustrated ...
447136,2108.01783,Re-examination of the Time Structure of the SN...,2021,Yuichi Oyama,"astro-ph.HE,hep-ex",The seven seconds' gap in the Kamiokande-II ...,Re-examination of the Time Structure of the SN...
210953,1601.00007,Merging binary black holes formed through chem...,2016,Ilya Mandel (University of Birmingham) and Sel...,"astro-ph.HE,astro-ph.SR",We explore a newly proposed channel to creat...,Merging binary black holes formed through chem...


In [23]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [24]:
df.iloc[0].categories

'cond-mat.stat-mech,cond-mat.soft'

## Train dataset creation

In [25]:
df_2, df_unused = train_test_split(df, train_size=0.6)  # take only 60% of dataset
df_train, df_test = train_test_split(df_2, train_size=0.8)

df.size, df_train.size, df_test.size

(700, 336, 84)

In [26]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

In [27]:
mlb = MultiLabelBinarizer()
# mlb.fit([[(k,v) for k, v in CATEGORIES.items()]]) #df_train['categories'])
mlb.fit([list(CATEGORIES.keys())]) #df_train['categories'])
mlb.classes_[:10]

array(['astro-ph', 'astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA',
       'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci'], dtype=object)

In [28]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]

    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    encoded_categories = mlb.transform([c.split(',') for c in examples['categories']]).astype(float)

    encoding["labels"] = encoded_categories

    return encoding

In [29]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']])
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']])
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

/Users/a.yushkovskiy/gh/atemate/redis-arXiv-search/.venv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [30]:
# Get inverse transform, as an example

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [('gr-qc',)]
Original categories gr-qc


In [32]:
# Store multilabel binarizer as a pickle file

!rm -rf checkpoint
!mkdir checkpoint
with open('checkpoint/mlb.pkl', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Training multi label class model

In [33]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [34]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 

# Define batch size according to your GPU RAM
batch_size = 8
nb_epochs = 1 # DEMONSTRATIVE PURPOSES
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    num_train_epochs=nb_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=1,
)

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    return multi_label_metrics(
        predictions=p.predictions, 
        labels=p.label_ids)

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

/Users/a.yushkovskiy/gh/atemate/redis-arXiv-search/.venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 48
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6
  Number of trainable parameters = 109599897
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
You're using a BertTokenizerFast tokeni

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.657666,0.026588,0.502474,0.000000


***** Running Evaluation *****
  Num examples = 12
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to paper-multilabel-finetuning/checkpoint-6
Configuration saved in paper-multilabel-finetuning/checkpoint-6/config.json
Model weights saved in paper-multilabel-finetuning/checkpoint-6/pytorch_model.bin
tokenizer config file saved in paper-multilabel-finetuning/checkpoint-6/tokenizer_config.json
Special tokens file saved in paper-multilabel-finetuning/checkpoint-6/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from paper-multilabel-finetuning/checkpoint-6 (score: 0.026587887740029542).


TrainOutput(global_step=6, training_loss=0.6783577601114908, metrics={'train_runtime': 9.3058, 'train_samples_per_second': 5.158, 'train_steps_per_second': 0.645, 'total_flos': 3161613275136.0, 'train_loss': 0.6783577601114908, 'epoch': 1.0})

In [37]:
eval_res = trainer.evaluate()
eval_res

***** Running Evaluation *****
  Num examples = 12
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.6576664447784424,
 'eval_f1': 0.026587887740029542,
 'eval_roc_auc': 0.5024737713970182,
 'eval_accuracy': 0.0,
 'eval_runtime': 0.6499,
 'eval_samples_per_second': 18.465,
 'eval_steps_per_second': 4.616,
 'epoch': 1.0}

## Perform inference on a given text sample

In [38]:
text = df['text'].iloc[5]
categories = df['categories'].iloc[5]
print(categories)

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

astro-ph.HE


In [39]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.3)] = 1

In [40]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

A Search for MeV to TeV Neutrinos from Fast Radio Bursts with IceCube   We present two searches for IceCube neutrino events coincident with 28 fast
radio bursts (FRBs) and one repeating FRB. The first improves upon a previous
IceCube analysis -- searching for spatial and temporal correlation of events
with FRBs at energies greater than roughly 50 GeV -- by increasing the
effective area by an order of magnitude. The second is a search for temporal
correlation of MeV neutrino events with FRBs. No significant correlation is
found in either search, therefore, we set upper limits on the time-integrated
neutrino flux emitted by FRBs for a range of emission timescales less than one
day. These are the first limits on FRB neutrino emission at the MeV scale, and
the limits set at higher energies are an order-of-magnitude improvement over
those set by any neutrino telescope.

[('astro-ph', 'astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA', 'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn',

In [41]:
trainer.save_model(output_dir='./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json


In [43]:
with open('./checkpoint/model_info.json', 'w') as f:
    f.write(json.dumps(eval_res, indent=4))